In [10]:
clear all
% build path
addpath(genpath('../Misc'))
build_path  niak psom zooniverse_qc  

# Set path
path_root = '/home/yassinebha/Drive/QC_zooniverse/';
#path_root = '/media/yassinebha/database29/Drive/QC_zooniverse/';
path_outline = [path_root 'template_layout/brain_outline_bold/'];
path_layout = [path_root 'template_layout/layout/'];

Adding library niak to the search path.

Adding library psom to the search path.

Adding library zooniverse_qc to the search path.



In [8]:
in.source = [path_root 'template_layout/test_data/HC0040013/anat_HC0040013_nuc_stereonl.nii.gz'];
out = [path_root 'template_layout/test_data/HC0040013/HC0040013_test_inv.nii.gz'];
opt.only_mask = false;
[in,out,opt] = zoo_brick_invert_contrast(in,out,opt);

out = /home/yassinebha/Drive/QC_zooniverse/template_layout/test_data/HC0040013/HC0040013_test_inv.nii.gz


In [28]:
# anat
clear in out opt
in.layout=[path_layout 'layout.nii.gz'];
out = '/home/yassinebha/Drive/QC_zooniverse/template_layout/test_anat_layout.nii.gz';
[in,out,opt]= zoo_brick_outline(in,out);


In [31]:
#func
clear in out opt
in.layout=[path_layout 'layout.nii.gz'];
in.mask_func = [path_outline 'func_mask_average_stereonl.mnc.gz'];
opt.modality = 'func';
out = '/home/yassinebha/Drive/QC_zooniverse/template_layout/test_func_layout.nii.gz';

[in,out,opt]= zoo_brick_outline(in,out,opt);

Transforming slices:.............................................................................................................................................................................................Done


In [29]:
in

in =

  scalar structure containing the fields:

    mask_func = 
    mask_anat = /home/yassinebha/git/niak/template/mni-models_icbm152-nl-2009-1.0/mni_icbm152_t1_tal_nlin_sym_09a_mask.mnc.gz
    target = 
    layout = /home/yassinebha/Drive/QC_zooniverse/template_layout/layout/layout.nii.gz

